# Overview
This notebook uses Google Cloud Platform's ML Vision API


[GCP Vision](https://cloud.google.com/vision/docs/libraries)
[GCP Big Query](https://googleapis.dev/python/bigquery/latest/index.html)

# Resources

https://cloud.google.com/solutions/creative-analysis-at-scale-with-google-cloud-and-machine-learning


## Bucket: 
https://console.cloud.google.com/storage/browser/huge-prototype-ads-images;tab=objects?project=data-science-dev-huge&prefix=&forceOnObjectsSortingFiltering=false

# GCP APIs

## Cloud Resource Manager API
https://console.cloud.google.com/apis/api/cloudresourcemanager.googleapis.com/overview?project=data-science-dev-huge

In [ ]:
!pip install --upgrade google-cloud-vision

     |████████████████████████████████| 471kB 8.0MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 


Requirement already up-to-date: google-cloud-vision in /usr/local/lib/python3.7/dist-packages (2.3.1)


In [ ]:
!pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in /usr/local/lib/python3.7/dist-packages (2.2.0)


In [ ]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=c2TqC5Q5bsNSTmYzrHETia6akMXiLo&prompt=consent&access_type=offline&code_challenge=ukVziRnrZTEja6SMD0OWbArMLft2KwO3VadHR8jTHzk&code_challenge_method=S256

Enter verification code: 4/1AY0e-g7O-mi8Dc_6FyH41L9Vo6BsYq7fSk3CBisB0fY7-RRYhAjsowxT5yg

You are now logged in as [mwebb@hugeinc.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [ ]:
!gcloud auth application-default login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=hUl2rnDvXBjcaXc1e3ORlKl9pGTMQn&prompt=consent&access_type=offline&code_challenge=bHVnHOEDLnjs6RaxAMbkw11MBAHNelqp6C5qUBAhKiQ&code_challenge_method=S256

Enter verification code: 4/1AY0e-g7njyyCC94lT_FMNGkddsa7dZzU8qj_eDMOuZE-P6dQeftshBmN5Y0

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot find a quota project to add to ADC. You might receive a "quota exceeded" or "API not enabled" error. Run $ gcloud auth application-defaul

In [ ]:
#1000 billing units per month before charge...cheap cheap though

# CONSTANTS
GCP_PROJECT = "data-science-dev-huge"
GCP_BUCKET = "huge-prototype-ads-images/creative_images_input_raw/"
#IMAGE = os.path.abspath(f'/content/{GCP_BUCKET}/citrix-banner-example.jpg')
#IMAGE_URL = f"gs://{GCP_BUCKET}/citrix-banner-example.jpg"
#IMAGE_URL2 = f"gs://{GCP_BUCKET}/022420_CTX_Workspace_Competitive_Pricing_Plan_160x600-f.jpg"

In [ ]:
# Imports
import io
import requests
import unicodedata
import pandas as pd
import os
from google.cloud import vision
import google.auth
from google.colab import auth
auth.authenticate_user()
credentials, project_id = google.auth.default()

/usr/local/lib/python3.7/dist-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/local/lib/python3.7/dist-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://clou

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# Authenticate to GCP.
!gcloud auth application-default login
!gcloud auth application-default set-quota-project $GCP_PROJECT
!gcloud config set project $GCP_PROJECT

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=8ksPAmoYv8qcaQeJvnjpgAAbwHnrtX&prompt=consent&access_type=offline&code_challenge=shtfKd8TMETxT8_u5aP0-J17AbsCgvRFGe3TuGZpnuU&code_challenge_method=S256

Enter verification code: 4/1AY0e-g6_vNS9y0izXX17DIInUnIKOVIezIU4kOtAkZVWxMvxv4cwx-jZDKs

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot find a quota project to add to ADC. You might receive a "quota exceeded" or "API not enabled" error. Run $ gcloud auth application-defaul

In [ ]:
#!cat /content/.config/application_default_credentials.json

{
  "client_id": "764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com",
  "client_secret": "d-FL95Q19q7MQmFpd7hHD0Ty",
  "quota_project_id": "data-science-dev-huge",
  "refresh_token": "1//04hsfj7wpmw9ZCgYIARAAGAQSNwF-L9IrIY_galLoMnZCixP2EmBU4nfgZLBsB_SjVDXrZK1qCYMceTWJRmi91jbNZXBd5GkCfS8",
  "type": "authorized_user"
}

### Creative Data Input Begins Here
1. Read in creative data
2. De-dupe creative data to a single name and ID and size combination
3. Persist these new IDs into drive.

In [ ]:
API = "https://s0.2mdn.net/"
ACCOUNT_ID = 9918507
CREATIVE_FILENAME = "bd_eng-AUS_edu_disp_wksp-1_overarchingexbenefit_728x90.jpg"

#We need to compile the URI, bring down the image from the URI, send image to GCP, send image to COLAB and store down on local Google Colab
#Compile the above to create the following api call:
#!wget https://s0.2mdn.net/9918507/022420_CTX_Workspace_Competitive_Pricing_Plan_160x600-f.jpg

!mkdir "/content/images/"
OUTPUT_LOC = f"/content/images/{CREATIVE_FILENAME}"
RAW_CREATIVE = f"https://s0.2mdn.net/{ACCOUNT_ID}/{CREATIVE_FILENAME}"


In [ ]:
RAW_CREATIVE 


'https://s0.2mdn.net/9918507/bd_eng-AUS_edu_disp_wksp-1_overarchingexbenefit_728x90.jpg'

In [ ]:
#r=requests.get(RAW_CREATIVE)

In [ ]:
#r.content
#file=open(dest_filename, "wb")
#file.write(r.content)
#file.close()

In [ ]:
# ##upload creatives from local (*** obviously this needs to change ***) -- need to get a reliable source of creatives - this is the bottleneck of entire system -- lots of attention here!!!
# import pandas as pd
# creatives = pd.read_csv("/content/drive/My Drive/creative_test_data/creative ID's with Names and KPI's_2.csv", header=None) ## this is from Rory and my export of DFA/DCM
# creatives = creatives.rename(columns={0: 'creative_filename'})
# ### this has a unicode character set at the end of each string that must be stripped

In [ ]:
# creatives.head()

,creative_filename
0,022420_CTX_Workspace_Competitive_Pricing_Plan_...
1,022420_CTX_Workspace_Competitive_Pricing_Plan_...
2,022420_CTX_Workspace_Competitive_Pricing_Plan_...
3,022420_CTX_Workspace_Competitive_Pricing_Plan_...
4,022420_CTX_Workspace_Competitive_Pricing_Plan_...


In [ ]:
# creatives.creative_filename[0]

'022420_CTX_Workspace_Competitive_Pricing_Plan_160x600-f.jpg\xa0'

In [ ]:
#creatives['creative_filename'].str.rfind.slice(stop=idx)
#creatives.apply(lambda x: x.creative_filename.rfind("_", 0, len(x.creative_filename, axis=1)))
# len(creatives.creative_filename)

64

In [ ]:
# #### this is key to match back to performance
# match_creative = []
# idx = creatives["creative_filename"].str.rfind("_") #reverse find catches the sizes of each creative
# for i in range(len(creatives.creative_filename)):
#   match_creative.append(creatives.iloc[i].creative_filename[:idx[i]])


In [ ]:
!gsutil ls gs://{GCP_BUCKET}

gs://huge-prototype-ads-images/creative_images_input_raw/
gs://huge-prototype-ads-images/creative_images_input_raw/021820_CTX_Workspace_Experience_SDWAN_OnDemand_Webinar_160x600-f.jpg
gs://huge-prototype-ads-images/creative_images_input_raw/021820_CTX_Workspace_Experience_SDWAN_OnDemand_Webinar_300x250-f.jpg
gs://huge-prototype-ads-images/creative_images_input_raw/08062020-163202805-082819_CTX_ADM_Cloud_Native_Application_Delivery_Webinar_OnDemand_160x600-f.jpg
gs://huge-prototype-ads-images/creative_images_input_raw/08062020-163205850-082819_CTX_ADM_Cloud_Native_Application_Delivery_Webinar_OnDemand_300x250-f.jpg
gs://huge-prototype-ads-images/creative_images_input_raw/081219_CTX_ADM_Hybrid_Multicloud_Webinar_OnDemand_300x250-f.jpg
gs://huge-prototype-ads-images/creative_images_input_raw/082819_CTX_ADM_Cloud_Native_Application_Delivery_Webinar_OnDemand_300x250-f.jpg
gs://huge-prototype-ads-images/creative_images_input_raw/728x90_B.jpg
gs://huge-prototype-ads-images/creative_images_inp

# START HERE
---



In [ ]:
GCP_PROJECT = "data-science-dev-huge"
GCP_BUCKET = "huge-prototype-ads-images/creative_images_input_raw/"

In [ ]:
imageList = !gsutil ls gs://{GCP_BUCKET}


In [ ]:
imageList

['gs://huge-prototype-ads-images/creative_images_input_raw/']

In [ ]:
#clean list
imageList = imageList[1:len(imageList)]

### the idea here is to find a way to de-dupe the creative names and their sizes so that we only send 1 image to vision opposed to all of the combinations of size

In [ ]:
# #this may not be necessary

# #remove all unicode strings - this can be done above
# for i in range(len(creatives.creative_filename)):
#   match_creative.append(creatives.iloc[i].creative_filename[:idx[i]])

In [ ]:
# set(match_creative)
imageList

['gs://huge-prototype-ads-images/creative_images_input_raw/021820_CTX_Workspace_Experience_SDWAN_OnDemand_Webinar_160x600-f.jpg',
 'gs://huge-prototype-ads-images/creative_images_input_raw/021820_CTX_Workspace_Experience_SDWAN_OnDemand_Webinar_300x250-f.jpg',
 'gs://huge-prototype-ads-images/creative_images_input_raw/08062020-163202805-082819_CTX_ADM_Cloud_Native_Application_Delivery_Webinar_OnDemand_160x600-f.jpg',
 'gs://huge-prototype-ads-images/creative_images_input_raw/08062020-163205850-082819_CTX_ADM_Cloud_Native_Application_Delivery_Webinar_OnDemand_300x250-f.jpg',
 'gs://huge-prototype-ads-images/creative_images_input_raw/081219_CTX_ADM_Hybrid_Multicloud_Webinar_OnDemand_300x250-f.jpg',
 'gs://huge-prototype-ads-images/creative_images_input_raw/082819_CTX_ADM_Cloud_Native_Application_Delivery_Webinar_OnDemand_300x250-f.jpg',
 'gs://huge-prototype-ads-images/creative_images_input_raw/728x90_B.jpg',
 'gs://huge-prototype-ads-images/creative_images_input_raw/ADC_Guide_DA-300x600.

In [ ]:
#I've come back to this line to consider the de-dupe process (end of day who gives a f if we throw a thousand through the system vs. 100....i'm probably the only guy)

## Only run this once as it is attempting to take image names and resolve the api and then storing them in the temp folder...this isn't what we want in end


In [ ]:
import unicodedata
import os
dest_filename = ""
command = ""
cnt = 0
RAW_CREATIVE = []
URL = []
response=""
creative_cleaned = []
for i in creatives['creative_filename'][0:25]:
  dest_filename = f'/content/images/{i}.jpg'
  RAW_CREATIVE = f"https://s0.2mdn.net/{ACCOUNT_ID}/{i}"
  creative_cleaned.append(unicodedata.normalize("NFKD", i))
  URL = unicodedata.normalize("NFKD", RAW_CREATIVE)
  print(URL)
  print(type(URL))
  response = requests.get(URL)
  print(response)
  file = open(dest_filename, "wb")
  file.write(response.content)
  file.close()
  cnt+=1


https://s0.2mdn.net/9918507/022420_CTX_Workspace_Competitive_Pricing_Plan_160x600-f.jpg 
<class 'str'>
<Response [200]>
https://s0.2mdn.net/9918507/022420_CTX_Workspace_Competitive_Pricing_Plan_300x250-f.jpg 
<class 'str'>
<Response [200]>
https://s0.2mdn.net/9918507/022420_CTX_Workspace_Competitive_Pricing_Plan_300x50-f.jpg 
<class 'str'>
<Response [200]>
https://s0.2mdn.net/9918507/022420_CTX_Workspace_Competitive_Pricing_Plan_300x600-f.jpg 
<class 'str'>
<Response [200]>
https://s0.2mdn.net/9918507/022420_CTX_Workspace_Competitive_Pricing_Plan_728x90-f.jpg 
<class 'str'>
<Response [200]>
https://s0.2mdn.net/9918507/bd_eng-AUS_edu_disp_wksp-1_overarchingexbenefit_160x600.jpg 
<class 'str'>
<Response [200]>
https://s0.2mdn.net/9918507/bd_eng-AUS_edu_disp_wksp-1_overarchingexbenefit_300x250.jpg 
<class 'str'>
<Response [200]>
https://s0.2mdn.net/9918507/bd_eng-AUS_edu_disp_wksp-1_overarchingexbenefit_300x600.jpg 
<class 'str'>
<Response [200]>
https://s0.2mdn.net/9918507/bd_eng-AUS_edu

In [ ]:
# Authenticate to GCP.
!gcloud auth application-default login
!gcloud auth application-default set-quota-project $GCP_PROJECT
!gcloud config set project $GCP_PROJECT

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=aO6jI7xfaeYaJIJmTaDemWc4LokHmN&prompt=consent&access_type=offline&code_challenge=PMTgmiIdQDNjF6hktChdAqMrhOOsL5O-ehYj4j9ldc4&code_challenge_method=S256

Enter verification code: 4/1AY0e-g5lN2IiDyjJRrW3tZZsXAeGXk3IE69siy48j-EO4FIEkHdYxlsr12w

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
/tools/google-cloud-sdk/lib/third_party/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials f

In [ ]:
#import unicodedata
#new_str = unicodedata.normalize("NFKD", t[1])
#new_str.strip()
#new_str
!mkdir /content/drive/My Drive/creative_test_data/

mkdir: cannot create directory ‘/content/drive/My’: Operation not supported
mkdir: cannot create directory ‘Drive/creative_test_data/’: No such file or directory


In [ ]:
!gsutil ls gs://huge-prototype-ads-images/creative_images_input_raw/ 

gs://huge-prototype-ads-images/creative_images_input_raw/
gs://huge-prototype-ads-images/creative_images_input_raw/021820_CTX_Workspace_Experience_SDWAN_OnDemand_Webinar_160x600-f.jpg
gs://huge-prototype-ads-images/creative_images_input_raw/021820_CTX_Workspace_Experience_SDWAN_OnDemand_Webinar_300x250-f.jpg
gs://huge-prototype-ads-images/creative_images_input_raw/08062020-163202805-082819_CTX_ADM_Cloud_Native_Application_Delivery_Webinar_OnDemand_160x600-f.jpg
gs://huge-prototype-ads-images/creative_images_input_raw/08062020-163205850-082819_CTX_ADM_Cloud_Native_Application_Delivery_Webinar_OnDemand_300x250-f.jpg
gs://huge-prototype-ads-images/creative_images_input_raw/081219_CTX_ADM_Hybrid_Multicloud_Webinar_OnDemand_300x250-f.jpg
gs://huge-prototype-ads-images/creative_images_input_raw/082819_CTX_ADM_Cloud_Native_Application_Delivery_Webinar_OnDemand_300x250-f.jpg
gs://huge-prototype-ads-images/creative_images_input_raw/728x90_B.jpg
gs://huge-prototype-ads-images/creative_images_inp

In [ ]:
!pwd

/content


In [ ]:
!gsutil cp gs://huge-prototype-ads-images/creative_images_input_raw/* .

Copying gs://huge-prototype-ads-images/creative_images_input_raw/021820_CTX_Workspace_Experience_SDWAN_OnDemand_Webinar_160x600-f.jpg...
Copying gs://huge-prototype-ads-images/creative_images_input_raw/021820_CTX_Workspace_Experience_SDWAN_OnDemand_Webinar_300x250-f.jpg...
Copying gs://huge-prototype-ads-images/creative_images_input_raw/08062020-163202805-082819_CTX_ADM_Cloud_Native_Application_Delivery_Webinar_OnDemand_160x600-f.jpg...
/ [3 files][170.8 KiB/170.8 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://huge-prototype-ads-images/creative_images_input_raw/08062020-163205850-082819_CTX_ADM_Cloud_Native_Application_Delivery_Webinar_OnDemand_300x250-f.jpg...
Copying gs://huge-prototype-ads-images/creative_images_inpu


### this needs attention...does imagery flow from colab to drive to storage? 

In [ ]:
!gsutil 

In [ ]:

### this needs attention...does imagery flow from colab to drive to storage? 

#!rm /content/images/*
#!cp -r /content/images/* /content/drive/My Drive/creative_test_data/
#!gsutil -m cp -r /content/drive/My Drive/creative_test_data/* gs://{$GCP_BUCKET}/
#!gsutil -m cp -r /content/drive/My Drive/creative_test_data/* gs://{$GCP_BUCKET}/

cp: target 'Drive/creative_test_data/' is not a directory
CommandException: No URLs matched: /content/drive/My
CommandException: No URLs matched: Drive/creative_test_data/*
CommandException: 2 files/objects could not be transferred.


In [ ]:
# bring in data from gcp - unneeded
#!gsutil cp -r gs://$GCP_BUCKET/ . 
#!ls $GCP_BUCKET/

Copying gs://huge-prototype-ads-images/022420_CTX_Workspace_Competitive_Pricing_Plan_160x600-f.jpg...
Copying gs://huge-prototype-ads-images/DyQ79iYUYAAWtNu.jpeg...
Copying gs://huge-prototype-ads-images/Screen Shot 2020-10-19 at 11.14.07 PM.png...
Copying gs://huge-prototype-ads-images/citrix-banner-example.jpg...
- [4 files][980.5 KiB/980.5 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://huge-prototype-ads-images/d400905.diff...
\ [5 files][982.7 KiB/982.7 KiB]                                                
Operation completed over 5 objects/982.7 KiB.                                    
 022420_CTX_Workspace_Competitive_Pricing_Plan_160x600-f.jpg
 citrix-banner-example.jpg
 d400905.diff
 DyQ79iYUYAAWtNu.jpeg
'Screen 

In [ ]:
#this copies data from colab to GCP -- run only for net new images
#!gsutil -m cp -r /content/images/* gs://$GCP_BUCKET/
#!gsutil ls gs://$GCP_BUCKET/

You are attempting to perform an operation that requires a project id, with none configured. Please re-run gsutil config and make sure to follow the instructions for finding and entering your default project id.


# Using the client proper with one feature per image

# Multiple Features in a Single Request But Each Request Can (and does) Contain Multiple Billable Units



In [ ]:
def process_image(url: str, feature_list: list=None):
    """
    Used for batch feature types
    """
    if not feature_list:
        feature_list = ['LABEL_DETECTION']
    if not isinstance(feature_list, list):
        feature_list = [feature_list]

    client = vision.ImageAnnotatorClient()

    features = [
        {'type_': f_type}
        for f_type in feature_list
    ]
    image = {'source': {'image_uri': url}}

    return client.annotate_image({
    'image': image,
    'features': features
    })

In [ ]:
# Example of running function with one feature
#response = process_image(IMAGE_URL)
#labels = response.label_annotations
#print('Object Labels:')
#for label in labels:
#    print(label.description)

In [ ]:
#IMAGE_URL2 = f"gs://{GCP_BUCKET}/022420_CTX_Workspace_Competitive_Pricing_Plan_160x600-f.jpg" # for this to really roll we need to isolate the creative name and the most important size and marry the two before vision

# Vision API hits here

In [ ]:
# Example for multiple -- add in the other modals here 
feature_list = [
    vision.Feature.Type.LABEL_DETECTION,
    vision.Feature.Type.TEXT_DETECTION,
    vision.Feature.Type.LOGO_DETECTION
]

In [ ]:
!gsutil ls gs://huge-prototype-ads-images/creative_images_input_raw

gs://huge-prototype-ads-images/creative_images_input_raw/


In [ ]:
!pip3 install google-cloud-storage

In [ ]:
import io, json
from google.cloud import vision_v1
from google.cloud.vision_v1 import AnnotateImageResponse
visionList = []
fList=[]
i=1
for f in imageList:
  annotatedImage = process_image(f)
  serialized_proto_plus = AnnotateImageResponse.serialize(annotatedImage)
  raw = AnnotateImageResponse.deserialize(serialized_proto_plus)
  response_json = AnnotateImageResponse.to_json(raw)
  response = json.loads(response_json)
  visionList.append(response)
  fList.append(f)



In [ ]:
len('gs://huge-prototype-ads-images/creative_images_input_raw/')

57

In [ ]:
fList

['gs://huge-prototype-ads-images/creative_images_input_raw/']

In [ ]:
#This is only for label annotations...we need to pull out logo and font here as well... 
 
gcp_files_w_labels = pd.DataFrame(columns=['image_index','label', 'score', 'topicality'])
for v in range(1,len(visionList)):
  for d in range(1,len(visionList[v]['labelAnnotations'])):
    gcp_files_w_labels = gcp_files_w_labels.append(
        dict(
        image_index=fList[v],
        label=visionList[v]['labelAnnotations'][d]['description'],
        score=visionList[v]['labelAnnotations'][d]['score'],
        topicality=visionList[v]['labelAnnotations'][d]['topicality']
      ), ignore_index=True)


IndentationError: ignored

In [ ]:
gcp_files_w_labels['image'] = gcp_files_w_labels.image_index.str[57:]

In [ ]:
gcp_files_w_labels.head()

,image_index,label,score,topicality,image
0,gs://huge-prototype-ads-images/creative_images...,Product,0.907342,0.907342,021820_CTX_Workspace_Experience_SDWAN_OnDemand...
1,gs://huge-prototype-ads-images/creative_images...,Sleeve,0.871849,0.871849,021820_CTX_Workspace_Experience_SDWAN_OnDemand...
2,gs://huge-prototype-ads-images/creative_images...,Sharing,0.868859,0.868859,021820_CTX_Workspace_Experience_SDWAN_OnDemand...
3,gs://huge-prototype-ads-images/creative_images...,Font,0.852063,0.852063,021820_CTX_Workspace_Experience_SDWAN_OnDemand...
4,gs://huge-prototype-ads-images/creative_images...,Smile,0.845074,0.845074,021820_CTX_Workspace_Experience_SDWAN_OnDemand...


In [ ]:
list(gcp_files_w_labels.label.unique())

['Product',
 'Sleeve',
 'Sharing',
 'Font',
 'Smile',
 'Output device',
 'T-shirt',
 'Adaptation',
 'People',
 'Yellow',
 'Building',
 'Technology',
 'Design',
 'Advertising',
 'Engineering',
 'Logo',
 'Brand',
 'Line',
 'Laptop',
 'Computer',
 'Circle',
 'Gesture',
 'Community',
 'Happy',
 'Mobile phone',
 'Communication Device',
 'Sky',
 'Electric blue',
 'Art',
 'Rectangle',
 'Graphics',
 'World',
 'Blue',
 'Azure',
 'Dress shirt',
 'Terrestrial plant',
 'Transport',
 'Parallel',
 'Slope',
 'Science',
 'Banner',
 'Travel',
 'Event',
 'Dog',
 'Personal computer',
 'Poster',
 'Companion dog',
 'Pattern',
 'Hand',
 'Arm',
 'Tableware',
 'Office supplies',
 'Organism']

In [ ]:
gcp_files_w_labels.to_csv('/content/drive/MyDrive/ImageAnnotator/images_labeled.csv', index = False)

## Connect DataFrame to Performance Data and Labeled Data THEN One Hot Encode Labels to Run

In [ ]:
!gsutil ls gs://huge-prototype-ads-images/creative_images_performance_input_raw/

gs://huge-prototype-ads-images/creative_images_performance_input_raw/
gs://huge-prototype-ads-images/creative_images_performance_input_raw/advertiser_filename_testout.csv
gs://huge-prototype-ads-images/creative_images_performance_input_raw/creative_images_performance_input_raw_creative_images_performance_input_raw_advertiser_filename_testout.csv


In [ ]:
!gsutil cp gs://huge-prototype-ads-images/creative_images_performance_input_raw/creative_images_performance_input_raw_creative_images_performance_input_raw_advertiser_filename_testout.csv . 

Copying gs://huge-prototype-ads-images/creative_images_performance_input_raw/creative_images_performance_input_raw_creative_images_performance_input_raw_advertiser_filename_testout.csv...
/ [1 files][  2.2 KiB/  2.2 KiB]                                                
Operation completed over 1 objects/2.2 KiB.                                      


In [ ]:
performance = pd.read_csv('/content/creative_images_performance_input_raw_creative_images_performance_input_raw_advertiser_filename_testout.csv')

In [ ]:

#df['image'] in performance['Filename']
#any(df['image'].isin(performance['Filename']))
sum(performance['Filename'].isin(gcp_files_w_labels['image']))

10

In [ ]:
#substring the df with the full 

In [ ]:
performance.shape

(23, 7)

# BOOM

In [ ]:
##this is miracle code ##
proper_1_hot = gcp_files_w_labels.set_index('image')['label'].str.get_dummies().max(level=0).reset_index()

In [ ]:
proper_1_hot

,image,Adaptation,Advertising,Arm,Art,Azure,Banner,Blue,Brand,Building,Circle,Communication Device,Community,Companion dog,Computer,Design,Dog,Dress shirt,Electric blue,Engineering,Event,Eyewear,Font,Gesture,Graphics,Hand,Happy,Laptop,Line,Logo,Mobile phone,Office supplies,Organism,Output device,Parallel,Pattern,People,Personal computer,Photo caption,Poster,Product,Rectangle,Science,Sharing,Sky,Sleeve,Slope,Smile,T-shirt,Tableware,Technology,Terrestrial plant,Transport,Travel,Vision care,World,Yellow
0,021820_CTX_Workspace_Experience_SDWAN_OnDemand...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0
1,08062020-163202805-082819_CTX_ADM_Cloud_Native...,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
2,08062020-163205850-082819_CTX_ADM_Cloud_Native...,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,081219_CTX_ADM_Hybrid_Multicloud_Webinar_OnDem...,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0
4,082819_CTX_ADM_Cloud_Native_Application_Delive...,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
5,728x90_B.jpg,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
6,ADC_Guide_DA-300x600.jpg,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
7,Banner_Vertical_Narrow_160x600.jpg,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
8,Citrix_Q2_ADC-Infographic_300x600.jpg,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0
9,Citrix_Q2_SD-WAN-Solution-Brief_300x600.jpg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
proper_1_hot.merge(performance, left_index=True, left_on='image', right_on='Filename')



,image,Adaptation,Advertising,Arm,Art,Azure,Banner,Blue,Brand,Building,Circle,Communication Device,Community,Companion dog,Computer,Design,Dog,Dress shirt,Electric blue,Engineering,Event,Eyewear,Font,Gesture,Graphics,Hand,Happy,Laptop,Line,Logo,Mobile phone,Office supplies,Organism,Output device,Parallel,Pattern,People,Personal computer,Photo caption,Poster,Product,Rectangle,Science,Sharing,Sky,Sleeve,Slope,Smile,T-shirt,Tableware,Technology,Terrestrial plant,Transport,Travel,Vision care,World,Yellow,Advertiser ID,Filename,Impressions,Clicks,CTR (Calc),Ga:Sessions,Engagement Score (Calc)
1,021820_CTX_Workspace_Experience_SDWAN_OnDemand...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,9918942,021820_CTX_Workspace_Experience_SDWAN_OnDemand...,31201.0,166.0,0.0053,115.0,7.1
6,08062020-163205850-082819_CTX_ADM_Cloud_Native...,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,9918942,08062020-163205850-082819_CTX_ADM_Cloud_Native...,84104.0,65.0,0.0008,27.0,8.8
2,081219_CTX_ADM_Hybrid_Multicloud_Webinar_OnDem...,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,9918942,081219_CTX_ADM_Hybrid_Multicloud_Webinar_OnDem...,33113.0,41.0,0.0012,10.0,4.3
7,728x90_B.jpg,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,9918942,728x90_B.jpg,440768.0,509.0,0.0012,111.0,22.9
8,ADC_Guide_DA-300x600.jpg,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,9918942,ADC_Guide_DA-300x600.jpg,127053.0,91.0,0.0007,60.0,21.8
13,Citrix_Q2_ADC-Infographic_300x600.jpg,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,9918942,Citrix_Q2_ADC-Infographic_300x600.jpg,4499.0,7.0,0.0016,4.0,7.0
14,Citrix_Q2_SD-WAN-Solution-Brief_300x600.jpg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,9918942,Citrix_Q2_SD-WAN-Solution-Brief_300x600.jpg,4198.0,6.0,0.0014,2.0,7.0
19,Cloud_Networking_Cloud_Network_Video_300x250.jpg,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9918942,Cloud_Networking_Cloud_Network_Video_300x250.jpg,79782.0,45.0,0.0006,28.0,11.7
18,Cloud_Networking_Cloud_eBook_300x600.jpg,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,9918942,Cloud_Networking_Cloud_eBook_300x600.jpg,82828.0,41.0,0.0005,39.0,13.4
21,dmd_de_disp_netw-1_7considerationsformicroapps...,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9918942,dmd_de_disp_netw-1_7considerationsformicroapps...,44904.0,41.0,0.0009,12.0,6.8


In [ ]:
joined = proper_1_hot.merge(performance, left_index=True, left_on='image', right_on='Filename')

In [ ]:
joined.to_csv('/content/drive/MyDrive/ImageAnnotator/labels_with_performance.csv', index=False)

# Start here (this is beginning marker for new script)

In [ ]:
joined = pd.read_csv('/content/drive/MyDrive/ImageAnnotator/labels_with_performance.csv')

# build regression on joined data surface best labels

In [ ]:
# grab columns that will always be present in the model a (all labels and target, drop all else)

colunmList = joined.columns.to_list()
kilCols = ['Advertiser ID', 'Filename', 'image']

In [ ]:
keepers = list(set(colunmList) - set(kilCols))


In [ ]:
mod_dat = joined[keepers]

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
y = mod_dat['CTR (Calc)']
X = mod_dat.drop('CTR (Calc)', axis=1)
reg = LinearRegression().fit(X, y)
#reg.score(X, y)
reg.coef_

array([ 2.46622288e-06,  3.87230135e-05, -2.19212988e-05, -1.35525272e-20,
        1.68728517e-05,  1.47013191e-05, -9.65287199e-06, -2.06774228e-05,
        1.35525272e-20, -1.06272732e-05,  0.00000000e+00, -2.35635023e-06,
       -1.38194904e-06, -1.46370510e-05,  1.47013191e-05,  3.87230135e-05,
        5.53243867e-05, -2.35635023e-06,  0.00000000e+00, -2.62782874e-06,
       -2.52655704e-05,  1.17960795e-06, -4.98417897e-06, -1.06272732e-05,
       -1.35217112e-05, -1.92934701e-05, -3.27509131e-05, -2.62782874e-06,
        1.34574430e-05,  3.20933755e-05,  1.22350962e-05,  1.44318442e-05,
        0.00000000e+00,  1.19690038e-05,  2.09598309e-05,  1.92292020e-05,
        0.00000000e+00,  0.00000000e+00,  9.74401192e-07,  1.22350962e-05,
       -2.42911693e-05, -6.03636848e-06,  0.00000000e+00, -2.87475251e-08,
        0.00000000e+00,  2.46622288e-06, -9.65287199e-06,  1.92292020e-05,
        5.09581097e-05, -2.62782874e-06,  0.00000000e+00, -3.58928436e-05,
       -2.35635023e-06, -

In [ ]:

coefs = pd.DataFrame(list(zip(reg.coef_, X)))


In [ ]:
# prune this and you're golden

coefs_df = coefs.sort_values(by=[0], ascending=False)
#print("this is the first one {}").format(coefs_sorted[1])

In [ ]:
for c in coefs_df[1]:
  print(f"create an image that has {c}")

create an image that has Advertising
create an image that has Event
create an image that has Travel
create an image that has Terrestrial plant
create an image that has Engagement Score (Calc)
create an image that has Clicks
create an image that has Computer
create an image that has Engineering
create an image that has Output device
create an image that has Laptop
create an image that has Brand
create an image that has Graphics
create an image that has Logo
create an image that has Sharing
create an image that has Mobile phone
create an image that has Pattern
create an image that has Science
create an image that has Ga:Sessions
create an image that has Sky
create an image that has Rectangle
create an image that has Circle
create an image that has People
create an image that has Transport
create an image that has Office supplies
create an image that has Arm
create an image that has Building
create an image that has Hand
create an image that has Parallel
create an image that has Design
cr

# Initial Prototype Complete:
## Next Steps:
1. Bring in OCR, Face, and Celebrity Detection
2. Determine where we should combine the above modules.
3. Determine the type of analysis plan (regression?, etc.)



In [ ]:
response = []
for f in files:
  response.append(process_image(IMAGE_URL, feature_list))
labels = response.label_annotations
texts = response.text_annotations
logo = response.logo_annotations
#l=labels[0].description
#t=texts[0].description
print('this is text output:')
print(texts) #the first desciptor can be split on /n and doesn't need a for loop
print('this is label output:') #this needs a for loop as each descriptor shows the next label
print(labels)
print('this is logo output:') #this needs a for loop as each descriptor shows the next label
print(logo)
### CALLS TO ACTION ARE PROBABLY VERBS BTW :)### - a money feature would be something like "detected call to action"

In [ ]:
#logo formatted
detected_logo=[]
for i in range(len(logo)):
  detected_logo.append(logo[i].description)

In [ ]:
detected_logo

['Citrix Systems', 'Citrix Systems']

In [ ]:
creative_id= [123]

In [ ]:
#d = {'Label':detected_labels,'Logo':detected_logo, 'Text': detected_text}
zippedList =  list(zip(creative_id, detected_logo, detected_labels, detected_text)) #gotta loop through detected text to grab it all...

In [ ]:
# this is useful output
set(zippedList)

{(123, 'Citrix Systems', 'Jeans', 'CİTRIX')}

In [ ]:
pd.DataFrame(set(zippedList))

,0,1,2,3
0,123,Citrix Systems,Jeans,CİTRIX


In [ ]:
detected_text

['CİTRIX', 'Synergy', 'Direct', '7®', '']

In [ ]:
#pd.DataFrame(d)
#pd.DataFrame({'creative_id': pd.Series(123),'Label': pd.Series(detected_labels), 'Logo': pd.Series(detected_logo)})
pd.DataFrame({'ID':pd.Series(creative_id), 'Label': pd.Series(detected_labels), 'Logo': pd.Series(detected_logo), 'Text': pd.Series(str(detected_text))}) # well dayum a proper data problem - organize me

,ID,Label,Logo,Text
0,123.0,Jeans,Citrix Systems,"['CİTRIX', 'Synergy', 'Direct', '7®', '']"
1,NaN,Denim,Citrix Systems,NaN
2,NaN,Photography,NaN,NaN
3,NaN,Job,NaN,NaN
4,NaN,Travel,NaN,NaN
5,NaN,Brand,NaN,NaN
6,NaN,T-shirt,NaN,NaN


In [ ]:
df = pd.DataFrame({'Label': pd.Series(detected_labels), 'Logo': pd.Series(detected_logo)})
df['ID'] = 123
df['text'] = str(detected_text)

In [ ]:
#flatten the lists?
#flat_list = [item for lab in detected_labels for item in lab]
#flat_list = [item for sublist in t for item in sublist]

In [ ]:
df.head()

### this should be reduced to 1 row per creative - it'll be a small dataframe anyway

,Label,Logo,ID,text
0,Jeans,Citrix Systems,123,"['CİTRIX', 'Synergy', 'Direct', '7®', '']"
1,Denim,Citrix Systems,123,"['CİTRIX', 'Synergy', 'Direct', '7®', '']"
2,Photography,NaN,123,"['CİTRIX', 'Synergy', 'Direct', '7®', '']"
3,Job,NaN,123,"['CİTRIX', 'Synergy', 'Direct', '7®', '']"
4,Travel,NaN,123,"['CİTRIX', 'Synergy', 'Direct', '7®', '']"


In [ ]:
# in order to match to creative name we must strip the sizing and filetype off of creative_filename

In [ ]:
# create dataframe keyed on creative by the vision output


In [ ]:
### marry performance data to each creative

In [ ]:
### interpretable model is the goal, correlation will suffice at first ###

In [ ]:
## Next task is to run through multiple examples and save down

## Legos are there, now we need to loop through each creative and have a dataframe as a result...

In [ ]:
joined_data= merge(source1, 2, 3, 4, 5, on_creative_id)